In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy.stats as stats
from scipy.stats import skew
import statsmodels.api as sm

%matplotlib inline

In [2]:
ACCRE_df = pd.read_csv('data/fullsample.csv')

In [3]:
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,14-00:00:00,11-01:32:50,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,13-18:00:00,12-09:26:41,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,13-18:00:00,13-08:27:36,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,13-18:00:00,12-03:50:59,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,13-18:00:00,11-08:43:46,1,1,production,0:0,2019-03-11T10:06:06


In [4]:
ACCRE_df.info()
# Notice 4,365,379 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4365379 entries, 0 to 4365378
Data columns (total 12 columns):
JOBID        object
ACCOUNT      object
USER         object
REQMEM       object
USEDMEM      object
REQTIME      object
USEDTIME     object
NODES        int64
CPUS         int64
PARTITION    object
EXITCODE     object
SUBMIT       object
dtypes: int64(2), object(10)
memory usage: 399.7+ MB


In [5]:
ACCRE_df.tail()
# No footer to remove

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
4365374,12741062_145,mickey,boyce,4096Mc,462.90M,00:10:00,00:01:44,1,1,production,0:0,2019-09-25T14:22:03
4365375,12741062_146,mickey,boyce,4096Mc,462.89M,00:10:00,00:01:44,1,1,production,0:0,2019-09-25T14:22:03
4365376,12741062_147,mickey,boyce,4096Mc,553.77M,00:10:00,00:01:53,1,1,production,0:0,2019-09-25T14:22:03
4365377,12741062_148,mickey,boyce,4096Mc,557.09M,00:10:00,00:01:52,1,1,production,0:0,2019-09-25T14:22:03
4365378,12741062_149,mickey,boyce,4096Mc,616.77M,00:10:00,00:01:52,1,1,production,0:0,2019-09-25T14:22:03


In [6]:
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,14-00:00:00,11-01:32:50,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,13-18:00:00,12-09:26:41,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,13-18:00:00,13-08:27:36,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,13-18:00:00,12-03:50:59,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,13-18:00:00,11-08:43:46,1,1,production,0:0,2019-03-11T10:06:06


In [7]:
ACCRE_df.dropna().head()
# This can be ignored because drop.duplicates is coming next

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,14-00:00:00,11-01:32:50,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,13-18:00:00,12-09:26:41,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,13-18:00:00,13-08:27:36,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,13-18:00:00,12-03:50:59,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,13-18:00:00,11-08:43:46,1,1,production,0:0,2019-03-11T10:06:06


In [8]:
ACCRE_df = ACCRE_df.drop_duplicates()

In [9]:
ACCRE_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3450799 entries, 0 to 4365378
Data columns (total 12 columns):
JOBID        object
ACCOUNT      object
USER         object
REQMEM       object
USEDMEM      object
REQTIME      object
USEDTIME     object
NODES        int64
CPUS         int64
PARTITION    object
EXITCODE     object
SUBMIT       object
dtypes: int64(2), object(10)
memory usage: 342.3+ MB


In [10]:
ACCRE_df_abridged = pd.read_csv('data/fullsample_dedup.csv')
# Reading in this csv is just to check if we eliminated the correct number of entries. 
# The ACCRE guy gave us this edited csv. 

In [11]:
ACCRE_df_abridged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3450799 entries, 0 to 3450798
Data columns (total 12 columns):
JOBID        object
ACCOUNT      object
USER         object
REQMEM       object
USEDMEM      object
REQTIME      object
USEDTIME     object
NODES        int64
CPUS         int64
PARTITION    object
EXITCODE     object
SUBMIT       object
dtypes: int64(2), object(10)
memory usage: 315.9+ MB


In [12]:
ACCRE_df = ACCRE_df.loc[ACCRE_df['PARTITION'] == 'production']
ACCRE_df.head()
# We want to ignore the partition named "debug"

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,14-00:00:00,11-01:32:50,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,13-18:00:00,12-09:26:41,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,13-18:00:00,13-08:27:36,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,13-18:00:00,12-03:50:59,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,13-18:00:00,11-08:43:46,1,1,production,0:0,2019-03-11T10:06:06


In [13]:
ACCRE_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3319257 entries, 0 to 4365378
Data columns (total 12 columns):
JOBID        object
ACCOUNT      object
USER         object
REQMEM       object
USEDMEM      object
REQTIME      object
USEDTIME     object
NODES        int64
CPUS         int64
PARTITION    object
EXITCODE     object
SUBMIT       object
dtypes: int64(2), object(10)
memory usage: 329.2+ MB


In [14]:
ACCRE_df.REQTIME = ACCRE_df.REQTIME.str.replace('-',':')
ACCRE_df.USEDTIME = ACCRE_df.USEDTIME.str.replace('-',':')
# REQTIME and USEDTIME are now manageable 

In [15]:
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,14:00:00:00,11:01:32:50,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,13:18:00:00,12:09:26:41,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,13:18:00:00,13:08:27:36,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,13:18:00:00,12:03:50:59,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,13:18:00:00,11:08:43:46,1,1,production,0:0,2019-03-11T10:06:06


In [16]:
date_time=ACCRE_df.REQTIME

In [17]:
%%time
# REQTIME needs to be in total seconds.
to_seconds = []
for time in date_time:
    split=time.split(':')
    if len(split)==4:
        seconds=int(split[0])*86400+int(split[1])*3600+int(split[2])*60+int(split[3])
        to_seconds.append(seconds)
    else: 
        seconds=int(split[0])*3600+int(split[1])*60+int(split[2])
        to_seconds.append(seconds)

CPU times: user 5.68 s, sys: 74.5 ms, total: 5.76 s
Wall time: 5.77 s


In [18]:
ACCRE_df.REQTIME = to_seconds
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,1209600,11:01:32:50,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,1188000,12:09:26:41,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,1188000,13:08:27:36,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,1188000,12:03:50:59,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,1188000,11:08:43:46,1,1,production,0:0,2019-03-11T10:06:06


In [19]:
used_time = ACCRE_df.USEDTIME

In [20]:
def convert_to_seconds(new):
    split_again=new.split(':')
    if len(split_again)==4: 
        sec_2=int(split_again[0])*86400+int(split_again[1])*3600+int(split_again[2])*60+int(split_again[3])
    
    else: 
        sec_2=int(split_again[0])*3600+int(split_again[1])*60+int(split_again[2])
               
    return sec_2

In [21]:
%%time 
ACCRE_df.USEDTIME = ACCRE_df.USEDTIME.apply(convert_to_seconds)

CPU times: user 5.64 s, sys: 168 ms, total: 5.81 s
Wall time: 5.62 s


In [22]:
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06


In [23]:
head=ACCRE_df.head()
# Just want to mess with a (very small) subset

In [24]:
head.REQMEM = head['REQMEM'].str.replace(r'(Mn|Mc)', r' \1')
head
# We need a space between values and units so we can split 

//anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808 Mn,115217.12M,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288 Mn,6516.10M,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288 Mn,0.09M,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288 Mn,8333.82M,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288 Mn,8304.03M,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06


In [25]:
head.USEDMEM = head['USEDMEM'].str.replace(r'(M)', r' \1')
head

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808 Mn,115217.12 M,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288 Mn,6516.10 M,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288 Mn,0.09 M,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288 Mn,8333.82 M,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288 Mn,8304.03 M,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06


In [26]:
#iterrows, itertuples?

In [27]:
head.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 12 columns):
JOBID        5 non-null object
ACCOUNT      5 non-null object
USER         5 non-null object
REQMEM       5 non-null object
USEDMEM      5 non-null object
REQTIME      5 non-null int64
USEDTIME     5 non-null int64
NODES        5 non-null int64
CPUS         5 non-null int64
PARTITION    5 non-null object
EXITCODE     5 non-null object
SUBMIT       5 non-null object
dtypes: int64(4), object(8)
memory usage: 520.0+ bytes


In [28]:
def convert_to_mc(df):
    num_unit=df.REQMEM.str.split(' ')
    
    df.REQMEM = num_unit.map(lambda x: int(x[0])) # changed to float, was int
    df['unit'] = num_unit.map(lambda x: x[1]) # made float
    idx = df.unit=='Mn'
    df.REQMEM.loc[idx]=df.REQMEM*df.NODES/df.CPUS
    
    return df

In [29]:
head=convert_to_mc(head)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i

In [30]:
head

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT,unit
0,6794220,laxtons,mikala,15488.0,115217.12 M,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44,Mn
1,6983193_1880,discovery,serina,12288.0,6516.10 M,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59,Mn
2,7055502_1863,discovery,serina,12288.0,0.09 M,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06,Mn
3,7055502_1800,discovery,serina,12288.0,8333.82 M,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06,Mn
4,7055502_1806,discovery,serina,12288.0,8304.03 M,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06,Mn


In [31]:
def convert_to_mc_M(df):
    num_unit=df.USEDMEM.str.split(' ')
    df.USEDMEM = num_unit.map(lambda x: float(x[0]))
    df['unit'] = num_unit.map(lambda x: x[1])
    idx = df.unit=='M'
    df.USEDMEM.loc[idx]=df.USEDMEM*df.NODES/df.CPUS
    
    return df

In [32]:
head.USEDMEM

0    115217.12 M
1      6516.10 M
2         0.09 M
3      8333.82 M
4      8304.03 M
Name: USEDMEM, dtype: object

In [33]:
head=convert_to_mc_M(head)
head

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT,unit
0,6794220,laxtons,mikala,15488.0,7201.07,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44,M
1,6983193_1880,discovery,serina,12288.0,6516.10,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59,M
2,7055502_1863,discovery,serina,12288.0,0.09,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06,M
3,7055502_1800,discovery,serina,12288.0,8333.82,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06,M
4,7055502_1806,discovery,serina,12288.0,8304.03,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06,M


In [34]:
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808 Mn,115217.12 M,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288 Mn,6516.10 M,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288 Mn,0.09 M,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288 Mn,8333.82 M,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288 Mn,8304.03 M,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06


In [35]:
ACCRE_df.USEDMEM.head()

0    115217.12 M
1      6516.10 M
2         0.09 M
3      8333.82 M
4      8304.03 M
Name: USEDMEM, dtype: object

In [36]:
ACCRE_df.REQMEM = ACCRE_df['REQMEM'].str.replace(r'(Mn|Mc)', r' \1')
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808 Mn,115217.12 M,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288 Mn,6516.10 M,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288 Mn,0.09 M,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288 Mn,8333.82 M,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288 Mn,8304.03 M,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06


In [37]:
# Recall that you have made this function: 
#def convert_to_mc(df):
    #num_unit=df.REQMEM.str.split(' ')
    
    #df.REQMEM = num_unit.map(lambda x: int(x[0]))
    #df['unit'] = num_unit.map(lambda x: x[1])
    #idx = df.unit=='Mn'
    #df.REQMEM.loc[idx]=df.REQMEM*df.NODES/df.CPUS
    
    #return df

In [38]:
ACCRE_df = convert_to_mc(ACCRE_df)
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT,unit
0,6794220,laxtons,mikala,247808.0,115217.12 M,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44,
1,6983193_1880,discovery,serina,12288.0,6516.10 M,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59,
2,7055502_1863,discovery,serina,12288.0,0.09 M,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06,
3,7055502_1800,discovery,serina,12288.0,8333.82 M,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06,
4,7055502_1806,discovery,serina,12288.0,8304.03 M,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06,


In [39]:
ACCRE_df.REQMEM.head()

0    247808.0
1     12288.0
2     12288.0
3     12288.0
4     12288.0
Name: REQMEM, dtype: float64